In [474]:
import requests
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import folium
from folium.plugins import MiniMap
from time import time
import pickle

Scrap live station status data in https://bikeshare.metro.net/stations/json/

In [475]:
def get_live_station(url):
    
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    stations = response.json()
    
    output = []
    
    for station in stations['features']:
        dict_keys = ['kioskId', 'bikesAvailable', 'docksAvailable', 'name', 'latitude', 'longitude']
        data = {k : station['properties'][k] for k in dict_keys}
        data['time'] = pd.to_datetime('today')
        output.append(data)

    return pd.DataFrame(output)

In [476]:
live_station_df = get_live_station("https://bikeshare.metro.net/stations/json/")
live_station_df.head()

,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,time
0,23,3,3005,34.04850,-118.25854,7th & Flower,2019-10-02 08:29:29.429393
1,15,9,3006,34.04554,-118.25667,Olive & 8th,2019-10-02 08:29:29.432897
2,6,14,3007,34.05048,-118.25459,5th & Grand,2019-10-02 08:29:29.433033
3,9,6,3008,34.04661,-118.26273,Figueroa & 9th,2019-10-02 08:29:29.433127
4,8,14,3010,34.03705,-118.25487,11th & Maple,2019-10-02 08:29:29.433230


In [477]:
# change time featur into str and take first 10 character.
# live_station_df.time = live_station_df.time.astype(str)
# live_station_df.time = live_station_df.time.str[:10]

Remove nano-seconds in time feature

In [478]:
live_station_df['time'] = live_station_df['time'].astype('datetime64[s]')

In [479]:
# convert unix timestamp to Y/M/D %H/%M/%S format
# live_station_df['time'] = pd.to_datetime(live_station_df['time'], unit='s')

In [480]:
# change name of time feature to ds and set it to index
# replace minutes and seconds to 0 
live_station_df.rename(columns={'time':'ds'}, inplace=True)
live_station_df.set_index('ds', inplace=True)
live_station_df.index = live_station_df.index.map(lambda x: x.replace(second=0))
live_station_df.index = live_station_df.index.map(lambda x: x.replace(minute=0))

In [481]:
live_station_df.kioskId = live_station_df.kioskId.astype(str)

In [482]:
live_station_df.head()

,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name
ds,,,,,,
2019-10-02 08:00:00,23,3,3005,34.04850,-118.25854,7th & Flower
2019-10-02 08:00:00,15,9,3006,34.04554,-118.25667,Olive & 8th
2019-10-02 08:00:00,6,14,3007,34.05048,-118.25459,5th & Grand
2019-10-02 08:00:00,9,6,3008,34.04661,-118.26273,Figueroa & 9th
2019-10-02 08:00:00,8,14,3010,34.03705,-118.25487,11th & Maple


### Create new column that concatenated with ds and kioskid to merge it with station_pred_df1

In [483]:
live_station_df.reset_index(inplace=True)

In [484]:
live_station_df['id'] = live_station_df.ds.astype(str) + '-' + live_station_df.kioskId
live_station_df.head()

,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,id
0,2019-10-02 08:00:00,23,3,3005,34.04850,-118.25854,7th & Flower,2019-10-02 08:00:00-3005
1,2019-10-02 08:00:00,15,9,3006,34.04554,-118.25667,Olive & 8th,2019-10-02 08:00:00-3006
2,2019-10-02 08:00:00,6,14,3007,34.05048,-118.25459,5th & Grand,2019-10-02 08:00:00-3007
3,2019-10-02 08:00:00,9,6,3008,34.04661,-118.26273,Figueroa & 9th,2019-10-02 08:00:00-3008
4,2019-10-02 08:00:00,8,14,3010,34.03705,-118.25487,11th & Maple,2019-10-02 08:00:00-3010


## Import forecasted station_pred_df1 that stored as a pickle

In [485]:
with open('station_pred_df1.pickle', 'rb') as f:
    station_pred_df1 = pickle.load(f)

station_pred_df1.head()

,station,ds,y_hat
0,3047.0,2019-09-24 13:00:00,0.439309
1,3047.0,2019-09-24 14:00:00,0.432759
2,3047.0,2019-09-24 15:00:00,0.434784
3,3047.0,2019-09-24 16:00:00,0.498354
4,3047.0,2019-09-24 17:00:00,0.608555


In [486]:
station_pred_df1.tail()

,station,ds,y_hat
17995,3078.0,2019-10-06 18:00:00,0.408277
17996,3078.0,2019-10-06 19:00:00,0.371387
17997,3078.0,2019-10-06 20:00:00,0.326630
17998,3078.0,2019-10-06 21:00:00,0.282961
17999,3078.0,2019-10-06 22:00:00,0.251276


In [487]:
# Take first 4 character in station feature
station_pred_df1.station = station_pred_df1.station.str[:4]

Create 

In [488]:
station_pred_df1['id'] = station_pred_df1.ds.astype(str) + '-' + station_pred_df1.station

In [489]:
station_pred_df1.tail()

,station,ds,y_hat,id
17995,3078,2019-10-06 18:00:00,0.408277,2019-10-06 18:00:00-3078
17996,3078,2019-10-06 19:00:00,0.371387,2019-10-06 19:00:00-3078
17997,3078,2019-10-06 20:00:00,0.326630,2019-10-06 20:00:00-3078
17998,3078,2019-10-06 21:00:00,0.282961,2019-10-06 21:00:00-3078
17999,3078,2019-10-06 22:00:00,0.251276,2019-10-06 22:00:00-3078


In [490]:
station_pred_df1.set_index('ds', inplace=True)

In [491]:
merge_live_pred_df = station_pred_df1.merge(live_station_df, on='id')
merge_live_pred_df.head()

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name
0,3047,0.422070,2019-10-02 08:00:00-3047,2019-10-02 08:00:00,18,18,3047,34.03998,-118.26640,Pico & Flower
1,3005,1.836403,2019-10-02 08:00:00-3005,2019-10-02 08:00:00,23,3,3005,34.04850,-118.25854,7th & Flower
2,3023,0.261683,2019-10-02 08:00:00-3023,2019-10-02 08:00:00,12,10,3023,34.05091,-118.24097,1st & Judge John Aiso
3,3051,0.339144,2019-10-02 08:00:00-3051,2019-10-02 08:00:00,7,9,3051,34.04542,-118.25352,7th & Broadway
4,3007,0.382885,2019-10-02 08:00:00-3007,2019-10-02 08:00:00,6,14,3007,34.05048,-118.25459,5th & Grand


In [492]:
merge_live_pred_df.head()

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name
0,3047,0.422070,2019-10-02 08:00:00-3047,2019-10-02 08:00:00,18,18,3047,34.03998,-118.26640,Pico & Flower
1,3005,1.836403,2019-10-02 08:00:00-3005,2019-10-02 08:00:00,23,3,3005,34.04850,-118.25854,7th & Flower
2,3023,0.261683,2019-10-02 08:00:00-3023,2019-10-02 08:00:00,12,10,3023,34.05091,-118.24097,1st & Judge John Aiso
3,3051,0.339144,2019-10-02 08:00:00-3051,2019-10-02 08:00:00,7,9,3051,34.04542,-118.25352,7th & Broadway
4,3007,0.382885,2019-10-02 08:00:00-3007,2019-10-02 08:00:00,6,14,3007,34.05048,-118.25459,5th & Grand


In [533]:
def normalize_demand_number(record):
    if record['y_hat'] >= 4.4:
        return round(record['y_hat']) * 3
    elif record['y_hat'] >= 4:
        return round(record['y_hat']) * 2
    elif record['y_hat'] >= 3:
        return round(record['y_hat']) * 2
    elif record['y_hat'] >= 1:
        return round(record['y_hat']) * 3
    elif record['y_hat'] >= 0.7:
        return round(record['y_hat']) * 2
    elif record['y_hat'] >= 0.4:
        return 1
    else:
        return 0

In [534]:
merge_live_pred_df['demand'] = merge_live_pred_df.apply(normalize_demand_number, axis=1)
merge_live_pred_df

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,demand
0,3047,0.422070,2019-10-02 08:00:00-3047,2019-10-02 08:00:00,18,18,3047,34.03998,-118.26640,Pico & Flower,1
1,3005,1.836403,2019-10-02 08:00:00-3005,2019-10-02 08:00:00,23,3,3005,34.04850,-118.25854,7th & Flower,6
2,3023,0.261683,2019-10-02 08:00:00-3023,2019-10-02 08:00:00,12,10,3023,34.05091,-118.24097,1st & Judge John Aiso,0
3,3051,0.339144,2019-10-02 08:00:00-3051,2019-10-02 08:00:00,7,9,3051,34.04542,-118.25352,7th & Broadway,0
4,3007,0.382885,2019-10-02 08:00:00-3007,2019-10-02 08:00:00,6,14,3007,34.05048,-118.25459,5th & Grand,0
5,3075,0.433229,2019-10-02 08:00:00-3075,2019-10-02 08:00:00,4,12,3075,34.04211,-118.25619,Broadway & 9th,1
6,3066,0.814117,2019-10-02 08:00:00-3066,2019-10-02 08:00:00,0,27,3066,34.06339,-118.23616,Spring & College,2
7,3082,1.016542,2019-10-02 08:00:00-3082,2019-10-02 08:00:00,12,15,3082,34.04652,-118.23741,Traction & Rose,3
8,3064,0.337102,2019-10-02 08:00:00-3064,2019-10-02 08:00:00,7,11,3064,34.04681,-118.25698,Grand & 7th,0
9,3052,0.384230,2019-10-02 08:00:00-3052,2019-10-02 08:00:00,5,29,3052,34.05110,-118.26456,7th & Bixel,0


In [537]:
with open('end_station_pred_df.pickle', 'rb') as f:
    end_station_pred_df = pickle.load(f)

end_station_pred_df.rename(columns={'y_hat': 'ys'}, inplace=True)
end_station_pred_df.head()

,station,ds,ys
0,3047.0,2017-04-01 07:00:00,0.339388
1,3047.0,2017-04-01 08:00:00,0.461941
2,3047.0,2017-04-01 09:00:00,0.454687
3,3047.0,2017-04-01 10:00:00,0.350759
4,3047.0,2017-04-01 11:00:00,0.236763


In [538]:
end_station_pred_df.station = end_station_pred_df.station.str[:4]
end_station_pred_df['id'] = end_station_pred_df.ds.astype(str) + '-' + end_station_pred_df.station
end_station_pred_df.set_index('ds', inplace=True)


In [540]:
merge_live_pred_df = merge_live_pred_df.merge(end_station_pred_df[['id', 'ys']], on='id')

In [543]:
def normalize_ys_number(record):
    if record['ys'] >= 3:
        return round(record['ys']) * 2
    elif record['ys'] >= 1:
        return round(record['ys']) * 3
    elif record['ys'] >= .8:
        return round(record['ys']) * 2
    elif record['ys'] >= .5:
        return round(record['ys']) * 1
    else:
        return 0

In [544]:
merge_live_pred_df['surplus'] = merge_live_pred_df.apply(normalize_ys_number, axis=1)
merge_live_pred_df

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,demand,ys,surplus
0,3047,0.422070,2019-10-02 08:00:00-3047,2019-10-02 08:00:00,18,18,3047,34.03998,-118.26640,Pico & Flower,1,0.325964,0
1,3005,1.836403,2019-10-02 08:00:00-3005,2019-10-02 08:00:00,23,3,3005,34.04850,-118.25854,7th & Flower,6,3.699468,8
2,3023,0.261683,2019-10-02 08:00:00-3023,2019-10-02 08:00:00,12,10,3023,34.05091,-118.24097,1st & Judge John Aiso,0,0.318803,0
3,3051,0.339144,2019-10-02 08:00:00-3051,2019-10-02 08:00:00,7,9,3051,34.04542,-118.25352,7th & Broadway,0,0.099961,0
4,3007,0.382885,2019-10-02 08:00:00-3007,2019-10-02 08:00:00,6,14,3007,34.05048,-118.25459,5th & Grand,0,1.211896,3
5,3075,0.433229,2019-10-02 08:00:00-3075,2019-10-02 08:00:00,4,12,3075,34.04211,-118.25619,Broadway & 9th,1,0.430229,0
6,3066,0.814117,2019-10-02 08:00:00-3066,2019-10-02 08:00:00,0,27,3066,34.06339,-118.23616,Spring & College,2,0.466051,0
7,3082,1.016542,2019-10-02 08:00:00-3082,2019-10-02 08:00:00,12,15,3082,34.04652,-118.23741,Traction & Rose,3,0.223523,0
8,3064,0.337102,2019-10-02 08:00:00-3064,2019-10-02 08:00:00,7,11,3064,34.04681,-118.25698,Grand & 7th,0,0.601296,1
9,3052,0.384230,2019-10-02 08:00:00-3052,2019-10-02 08:00:00,5,29,3052,34.05110,-118.26456,7th & Bixel,0,0.299696,0


In [545]:
def nexthour(record):
    if record['bikesAvailable'] - record['demand'] + record['surplus'] >= 0:
        return record['bikesAvailable'] - record['demand'] + record['surplus']
    else:
        return 0

In [547]:
merge_live_pred_df['nextHour'] = merge_live_pred_df.apply(nexthour, axis=1)
merge_live_pred_df

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,demand,ys,surplus,nextHour
0,3047,0.422070,2019-10-02 08:00:00-3047,2019-10-02 08:00:00,18,18,3047,34.03998,-118.26640,Pico & Flower,1,0.325964,0,17
1,3005,1.836403,2019-10-02 08:00:00-3005,2019-10-02 08:00:00,23,3,3005,34.04850,-118.25854,7th & Flower,6,3.699468,8,25
2,3023,0.261683,2019-10-02 08:00:00-3023,2019-10-02 08:00:00,12,10,3023,34.05091,-118.24097,1st & Judge John Aiso,0,0.318803,0,12
3,3051,0.339144,2019-10-02 08:00:00-3051,2019-10-02 08:00:00,7,9,3051,34.04542,-118.25352,7th & Broadway,0,0.099961,0,7
4,3007,0.382885,2019-10-02 08:00:00-3007,2019-10-02 08:00:00,6,14,3007,34.05048,-118.25459,5th & Grand,0,1.211896,3,9
5,3075,0.433229,2019-10-02 08:00:00-3075,2019-10-02 08:00:00,4,12,3075,34.04211,-118.25619,Broadway & 9th,1,0.430229,0,3
6,3066,0.814117,2019-10-02 08:00:00-3066,2019-10-02 08:00:00,0,27,3066,34.06339,-118.23616,Spring & College,2,0.466051,0,0
7,3082,1.016542,2019-10-02 08:00:00-3082,2019-10-02 08:00:00,12,15,3082,34.04652,-118.23741,Traction & Rose,3,0.223523,0,9
8,3064,0.337102,2019-10-02 08:00:00-3064,2019-10-02 08:00:00,7,11,3064,34.04681,-118.25698,Grand & 7th,0,0.601296,1,8
9,3052,0.384230,2019-10-02 08:00:00-3052,2019-10-02 08:00:00,5,29,3052,34.05110,-118.26456,7th & Bixel,0,0.299696,0,5


In [259]:
# df = merge_live_pred_df.copy()
# df.to_csv('df_9_30_5pm.csv')

In [513]:
!ls

Data                             end_station_pred_df.pickle
LICENSE                          functions.py
Live_station_status.ipynb        history
README.md                        markers.html
TimeSeries_BikeStation.ipynb     station_name.csv
capstone_EDA.ipynb               station_name.json
capstone_TimeSeries-Copy1.ipynb  station_pred_df.pickle
capstone_TimeSeries.ipynb        station_pred_df1.pickle
capstone_TimeSeries_hourly.ipynb


## Create Folium Map that will display available bikes,  open docks, and forecasted available bikes in next hour for each bike stations. 

In [275]:
def get_base_map(df):
    minimap = MiniMap(toggle_display=True, width=300, height=150, zoom_animation=True, zoom_level_offset=-6, position='topleft')
   
    return folium.Map(location=[df.latitude.mean(),
                             df.longitude.mean()],
                   zoom_start=20,
                     tiles='OpenStreetMap').add_child(minimap)
base = get_base_map(merge_live_pred_df)
base

In [264]:
def add_station_markers(initial_map, df):
    #station location visualization
    
    out_map = initial_map
    for lat, lon, Name, Available, Opendocks, Next_hr_remaining, in zip(df['latitude'], df['longitude'], df['name'], 
                                df['bikesAvailable'], df['docksAvailable'], df['nextHour']):
        folium.Marker([lat,lon], popup=(str(Name).capitalize() + '<br>'
                                        '<br><b>Available: </b>' + str(Available) + '<br>'
                                        '<br><b>Opendocks: </b>' + str(Opendocks) + '<br>'
                                        '<br><b>Remaining Next Hour:</b> ' + str(Next_hr_remaining)),
                     icon=folium.Icon(color='green')).add_to(out_map)
    return out_map

markers = add_station_markers(base, merge_live_pred_df)
markers 

In [257]:
markers.save('markers.html')

In [172]:
!ls

Data                             capstone_TimeSeries.ipynb
LICENSE                          capstone_TimeSeries_hourly.ipynb
Live_station_status.ipynb        functions.py
README.md                        markers.html
TimeSeries_BikeStation.ipynb     station_pred_df.pickle
capstone_EDA.ipynb               station_pred_df1.pickle
capstone_TimeSeries-Copy1.ipynb


In [458]:
live_station_df.head()


,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,id
0,2019-09-30 17:00:00,20,6,3005,34.04850,-118.25854,7th & Flower,2019-09-30 17:00:00-3005
1,2019-09-30 17:00:00,18,5,3006,34.04554,-118.25667,Olive & 8th,2019-09-30 17:00:00-3006
2,2019-09-30 17:00:00,4,18,3007,34.05048,-118.25459,5th & Grand,2019-09-30 17:00:00-3007
3,2019-09-30 17:00:00,5,10,3008,34.04661,-118.26273,Figueroa & 9th,2019-09-30 17:00:00-3008
4,2019-09-30 17:00:00,6,16,3010,34.03705,-118.25487,11th & Maple,2019-09-30 17:00:00-3010


In [367]:
live_station_df.groupby('name')['latitude','longitude'].apply(lambda x: x.values.tolist()).to_dict()

{'11th & Maple': [[34.03705, -118.25487]],
 '12th & Hill': [[34.03861, -118.26086]],
 '12th & Valencia': [[34.04557, -118.27419]],
 '17th St / SMC Expo Line Station': [[34.02338, -118.47964]],
 '18th & Figueroa': [[34.03568, -118.27081]],
 '18th & San Pedro': [[34.02851, -118.25667]],
 '1st & Central': [[34.0493, -118.23881]],
 '1st & Judge John Aiso': [[34.05091, -118.24097]],
 '23rd St & Flower': [[34.03027, -118.27319]],
 '25th & Vermont': [[34.03364, -118.29131]],
 '28th & Figueroa': [[34.02629, -118.27769]],
 '28th & University': [[34.02762, -118.28068]],
 '29th & Ellendale': [[34.02835, -118.28867]],
 '2nd & Figueroa': [[34.05697, -118.25359]],
 '2nd & Hill': [[34.05287, -118.24749]],
 '32nd & Figueroa': [[34.02371, -118.27917]],
 '3rd & San Pedro': [[34.04775, -118.24317]],
 '3rd & Santa Fe': [[34.04607, -118.23309]],
 '4th & Vermont': [[34.06699, -118.29088]],
 '5th & Grand': [[34.05048, -118.25459]],
 '5th & Hewitt': [[34.04169, -118.23535]],
 '7th & Berendo': [[34.05969, -118

In [350]:
d = {}
for i in df['name'].unique():
    d[i] = [{live_station_df['name'][j]: (live_station_df['latitude'][j], live_station_df['longitude'][j])} for j in 
            live_station_df[live_station_df['name']==i].index]

In [355]:
d

{'Pico & Flower': [{'Pico & Flower': (34.03998, -118.2664)}],
 '7th & Flower': [{'7th & Flower': (34.0485, -118.25854)}],
 '1st & Judge John Aiso': [{'1st & Judge John Aiso': (34.05091, -118.24097)}],
 '7th & Broadway': [{'7th & Broadway': (34.04542, -118.25352)}],
 '5th & Grand': [{'5th & Grand': (34.05048, -118.25459)}],
 'Broadway & 9th': [{'Broadway & 9th': (34.04211, -118.25619)}],
 'Spring & College': [{'Spring & College': (34.06339, -118.23616)}],
 'Traction & Rose': [{'Traction & Rose': (34.04652, -118.23741)}],
 'Grand & 7th': [{'Grand & 7th': (34.04681, -118.25698)}],
 '7th & Bixel': [{'7th & Bixel': (34.0511, -118.26456)}],
 'Main & 4th': [{'Main & 4th': (34.04885, -118.24642)}],
 'Olive & 8th': [{'Olive & 8th': (34.04554, -118.25667)}],
 'Grand & Olympic': [{'Grand & Olympic': (34.04373, -118.26014)}],
 'Figueroa & 8th': [{'Figueroa & 8th': (34.0484, -118.26095)}],
 'Hope & 6th': [{'Hope & 6th': (34.04989, -118.25588)}],
 'Main & 1st': [{'Main & 1st': (34.05194, -118.24353)

In [466]:
station_name_df = merge_live_pred_df[['station', 'name', 'latitude', 'longitude']]
station_name_df.shape

(56, 4)

In [473]:
station_name_df.head()

,station,name,latitude,longitude
0,3047,Pico & Flower,34.03998,-118.26640
1,3005,7th & Flower,34.04850,-118.25854
2,3023,1st & Judge John Aiso,34.05091,-118.24097
3,3051,7th & Broadway,34.04542,-118.25352
4,3007,5th & Grand,34.05048,-118.25459


In [467]:
station_name_df.to_csv("station_name.csv")

In [470]:
import csv

with open('station_name.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        stations = {}
        for row in reader:
            kiosk_id= row['station']
            lat = row['latitude']
            lng = row['longitude']
            stations[kiosk_id] = (lat, lng)


In [471]:
import json

with open('station_name.json', 'w') as outfile:
    json.dump(stations, outfile)

In [472]:
!ls

Data                             df.csv
LICENSE                          df_9_26_6pm.csv
Live_station_status.ipynb        df_9_30_5pm.csv
README.md                        functions.py
TimeSeries_BikeStation.ipynb     markers.html
capstone_EDA.ipynb               station_name.csv
capstone_TimeSeries-Copy1.ipynb  station_name.json
capstone_TimeSeries.ipynb        station_pred_df.pickle
capstone_TimeSeries_hourly.ipynb station_pred_df1.pickle


In [445]:
# station_name_df.to_csv("station_name.csv")

In [454]:
# # data = request.form
# # print(data)

# # station = data["station"]

# stations = station_name_df

# for lat, lng = (stations['latitude'][0], stations['longitude'][0])

In [465]:
test = merge_live_pred_df[['station', 'name', 'latitude', 'longitude']]
test.shape

(56, 4)